In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_forecast_component
from sqlalchemy import create_engine
import psycopg2
import tensorflow as tf

df = pd.read_csv('../final_data.csv')
df['start_time'] = pd.to_datetime(df['start_time'])
df = df.sort_values('start_time').reset_index(drop = True)
df.head()

,start_time,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
0,2013-06-27 01:00:00,1,2,22.5,87,20.2,24.6,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,2013-06-27 11:00:00,1,8,26.5,72,21.0,31.9,0.1,0.1,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2013-06-27 11:00:00,3,9,26.5,72,21.0,31.9,0.1,0.1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2013-06-27 12:00:00,1,14,27.2,70,21.2,33.2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2013-06-27 12:00:00,2,20,27.2,70,21.2,33.2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
n = len(df)
train = df[0:int(n*0.7)]
val = df[int(n*0.7):int(n*0.9)]
test = df[int(n*0.9):]

In [30]:
train

,start_time,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
0,2013-06-27 01:00:00,1,2,22.5,87,20.2,24.6,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,2013-06-27 11:00:00,1,8,26.5,72,21.0,31.9,0.1,0.1,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2013-06-27 11:00:00,3,9,26.5,72,21.0,31.9,0.1,0.1,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2013-06-27 12:00:00,1,14,27.2,70,21.2,33.2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2013-06-27 12:00:00,2,20,27.2,70,21.2,33.2,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036194,2018-04-09 11:00:00,1,2,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036195,2018-04-09 11:00:00,2,4,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036196,2018-04-09 11:00:00,5,3,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1036197,2018-04-09 11:00:00,5,5,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
val

,start_time,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
1036199,2018-04-09 11:00:00,1,3,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036200,2018-04-09 11:00:00,3,1,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036201,2018-04-09 11:00:00,3,8,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036202,2018-04-09 11:00:00,4,11,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1036203,2018-04-09 11:00:00,2,6,2.6,77,-1.1,0.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332251,2019-06-14 08:00:00,151,12,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1332252,2019-06-14 08:00:00,147,14,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1332253,2019-06-14 08:00:00,79,20,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1332254,2019-06-14 08:00:00,93,8,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
test

,start_time,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,...,60649,60651,60653,60654,60657,60659,60660,60661,60696,60804
1332256,2019-06-14 08:00:00,61,2,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1332257,2019-06-14 08:00:00,1,0,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1332258,2019-06-14 08:00:00,15,12,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1332259,2019-06-14 08:00:00,75,3,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1332260,2019-06-14 08:00:00,4,4,16.3,64,9.6,14.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480280,2019-12-31 23:00:00,1,8,-2.5,75,-6.4,-7.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1480281,2019-12-31 23:00:00,3,9,-2.5,75,-6.4,-7.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1480282,2019-12-31 23:00:00,3,2,-2.5,75,-6.4,-7.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1480283,2019-12-31 23:00:00,3,12,-2.5,75,-6.4,-7.9,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df.start_time.dtype

dtype('<M8[ns]')

In [3]:
df.columns

Index(['start_time', 'trips', 'landmarks', 'temp', 'rel_humidity', 'dewpoint',
       'apparent_temp', 'precip', 'rain', 'snow', 'cloudcover', 'windspeed',
       '60201', '60202', '60208', '60301', '60302', '60304', '60601', '60602',
       '60603', '60604', '60605', '60606', '60607', '60608', '60609', '60610',
       '60611', '60612', '60613', '60614', '60615', '60616', '60617', '60618',
       '60619', '60620', '60621', '60622', '60623', '60624', '60625', '60626',
       '60628', '60629', '60630', '60632', '60636', '60637', '60638', '60640',
       '60641', '60642', '60643', '60644', '60645', '60646', '60647', '60649',
       '60651', '60653', '60654', '60657', '60659', '60660', '60661', '60696',
       '60804'],
      dtype='object')

## Find number of hours since start

In [4]:
start = df['start_time'][0]
df['hours_since_start'] = (df['start_time'] - start).dt.total_seconds()/3600
df_new = df.drop(labels='start_time', axis = 1)
df_new.head()

,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,...,60651,60653,60654,60657,60659,60660,60661,60696,60804,hours_since_start
0,1,2,22.5,87,20.2,24.6,0.0,0.0,0.0,34,...,0,0,0,0,0,0,1,0,0,0.0
1,3,2,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,0,0,0,1,0,0,29.0
2,4,8,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,29.0
3,1,14,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,29.0
4,1,21,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,0,0,0,0,0,0,29.0


In [5]:
df_new.hours_since_start

0              0.0
1             29.0
2             29.0
3             29.0
4             29.0
            ...   
1480280    46994.0
1480281    47666.0
1480282    47666.0
1480283    48146.0
1480284    31514.0
Name: hours_since_start, Length: 1480285, dtype: float64

## Yearly and Weekly and Daily signals

In [6]:
df_new['Year sin'] = np.sin(df_new['hours_since_start'] * (2 * np.pi / (365*24)))
df_new['Year cos'] = np.cos(df_new['hours_since_start'] * (2 * np.pi / (365*24)))

df_new['Week sin'] = np.sin(df_new['hours_since_start'] * (2 * np.pi / (7*24)))
df_new['Week cos'] = np.cos(df_new['hours_since_start'] * (2 * np.pi / (7*24)))

df_new['Day sin'] = np.sin(df_new['hours_since_start'] * (2 * np.pi / (24)))
df_new['Day cos'] = np.cos(df_new['hours_since_start'] * (2 * np.pi / (24)))

## Split

In [7]:
column_indices = {name: i for i, name in enumerate(df_new.columns)}

n = len(df_new)
train_df = df_new[0:int(n*0.7)]
val_df = df_new[int(n*0.7):int(n*0.9)]
test_df = df_new[int(n*0.9):]

num_features = df_new.shape[1]
train_df.head()

,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,...,60661,60696,60804,hours_since_start,Year sin,Year cos,Week sin,Week cos,Day sin,Day cos
0,1,2,22.5,87,20.2,24.6,0.0,0.0,0.0,34,...,1,0,0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000
1,3,2,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,1,0,0,29.0,0.020799,0.999784,0.884115,0.467269,0.965926,0.258819
2,4,8,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,29.0,0.020799,0.999784,0.884115,0.467269,0.965926,0.258819
3,1,14,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,29.0,0.020799,0.999784,0.884115,0.467269,0.965926,0.258819
4,1,21,22.8,67,16.4,21.5,0.0,0.0,0.0,7,...,0,0,0,29.0,0.020799,0.999784,0.884115,0.467269,0.965926,0.258819


In [8]:
# train_mean = train_df.mean()
# train_std = train_df.std()

# train_df = (train_df - train_mean) / train_std
# val_df = (val_df - train_mean) / train_std
# test_df = (test_df - train_mean) / train_std

## Data Window

- Input window: length of given history (i.e. length in time of training data)
- Shift: make prediction n units of time in the future
- Label width: # of predictions made in the future

In [9]:
class WindowGenerator():
    '''
    - Input width: length of given history (i.e. length in time of training data)
    - Shift: make prediction n units of time in the future
    - Label width: # of predictions made in the future

    total_window_size, input_indicies, label_indices are attributes of WindowGenerator object
    '''

    def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                            enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

def plot(self, model=None, plot_col='trip_count', max_subplots=3, plot_title=None):
    ''' 
    Plots inputs, labels, predictions
    '''
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        if n == 0 and plot_title is not None:
            plt.title(plot_title)
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

            # print(predictions[n, :, label_col_index])

        if n == 0:
            plt.legend()
    
    plt.xlabel('Day')

WindowGenerator.plot = plot

def split_window(self, features):
    '''
    Converts total window into a window of inputs and a window of labels
    '''
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

## Create Dataset

In [10]:
def make_dataset(self, data):
    ''' 
    Takes time series dataset and turns it into tf.data.Dataset of (input_window, label_window)
    '''
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [11]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example


In [12]:
test_df

,trips,landmarks,temp,rel_humidity,dewpoint,apparent_temp,precip,rain,snow,cloudcover,...,60661,60696,60804,hours_since_start,Year sin,Year cos,Week sin,Week cos,Day sin,Day cos
1332256,2,4,21.0,84,18.2,22.3,0.3,0.3,0.0,65,...,0,0,0,54680.0,0.998740,0.050187,0.149042,-0.988831,0.866025,-0.500000
1332257,40,3,21.0,84,18.2,22.3,0.3,0.3,0.0,65,...,0,0,0,54680.0,0.998740,0.050187,0.149042,-0.988831,0.866025,-0.500000
1332258,9,12,21.0,84,18.2,22.3,0.3,0.3,0.0,65,...,0,0,0,54680.0,0.998740,0.050187,0.149042,-0.988831,0.866025,-0.500000
1332259,5,14,21.0,84,18.2,22.3,0.3,0.3,0.0,65,...,0,0,0,54680.0,0.998740,0.050187,0.149042,-0.988831,0.866025,-0.500000
1332260,8,11,21.0,84,18.2,22.3,0.3,0.3,0.0,65,...,0,0,0,54680.0,0.998740,0.050187,0.149042,-0.988831,0.866025,-0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480280,1,0,9.3,93,8.2,6.6,0.0,0.0,0.0,98,...,0,0,0,46994.0,0.751722,-0.659481,-0.988831,-0.149042,0.500000,0.866025
1480281,1,0,-0.2,78,-3.5,-5.6,0.0,0.0,0.0,81,...,0,0,0,47666.0,0.360376,-0.932807,-0.988831,-0.149042,0.500000,0.866025
1480282,2,2,-0.2,78,-3.5,-5.6,0.0,0.0,0.0,81,...,0,0,0,47666.0,0.360376,-0.932807,-0.988831,-0.149042,0.500000,0.866025
1480283,1,2,-2.3,62,-8.5,-8.1,0.0,0.0,0.0,0,...,0,0,0,48146.0,0.024384,-0.999703,-0.500000,-0.866025,0.500000,0.866025


In [19]:
# takes 231 days of data, forecasts next 24 hours.
hours_into_future = 24
w1 = WindowGenerator(input_width=30*24, label_width=hours_into_future, shift=hours_into_future,
                     label_columns=['trips'])
w1

Total window size: 744
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 

In [20]:
w1.train.element_spec

(TensorSpec(shape=(None, 720, 75), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 24, 1), dtype=tf.float32, name=None))

In [21]:
for example_inputs, example_labels in w1.train.take(2):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 720, 75)
Labels shape (batch, time, features): (32, 24, 1)
Inputs shape (batch, time, features): (32, 720, 75)
Labels shape (batch, time, features): (32, 24, 1)


In [33]:
for i in w1.train.take(3):
    # len(i[0][0][0])
    print(i[0][0][0])

tf.Tensor(
[ 1.   2.  22.5 87.  20.2 24.6  0.   0.   0.  34.   6.8  0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.
  1.   0.   1.   0.   1. ], shape=(75,), dtype=float32)
tf.Tensor(
[ 3.0000000e+00  1.0000000e+01  2.4100000e+01  6.2000000e+01
  1.6299999e+01  2.4000000e+01  0.0000000e+00  0.0000000e+00
  0.0000000e+00  9.0000000e+00  1.5100000e+01  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  1.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000

In [22]:
OUT_STEPS = hours_into_future

multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
    # Shape => [batch, out_steps, features].
    
])


lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(50, return_sequences=True),
    

    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])


In [23]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=patience,
                                                        mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                    metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                      callbacks=[early_stopping])
    
    return history

In [24]:
history = compile_and_fit(multi_lstm_model, w1)


Epoch 1/20
32358/32358 [==============================] - 679s 21ms/step - loss: 572.6321 - mean_absolute_error: 14.1235 - val_loss: 832.6542 - val_mean_absolute_error: 18.8956
Epoch 2/20
32358/32358 [==============================] - 680s 21ms/step - loss: 573.3433 - mean_absolute_error: 14.1294 - val_loss: 832.0500 - val_mean_absolute_error: 18.8474
Epoch 3/20
32358/32358 [==============================] - 678s 21ms/step - loss: 573.1873 - mean_absolute_error: 14.1706 - val_loss: 831.8995 - val_mean_absolute_error: 18.8352
Epoch 4/20
32358/32358 [==============================] - 674s 21ms/step - loss: 572.8408 - mean_absolute_error: 14.1939 - val_loss: 831.8942 - val_mean_absolute_error: 18.8346
Epoch 5/20
 2626/32358 [=>............................] - ETA: 9:05 - loss: 171.0491 - mean_absolute_error: 8.5785

KeyboardInterrupt: 